<center><p float="center">
  <img src="https://mma.prnewswire.com/media/1458111/Great_Learning_Logo.jpg?p=facebook" width="200" height="100"/>
</p></center>

<h1><center><font size=10> Generative AI for NLP Program</center></font></h1>
<h1><center> Project </center></h1>

# **GA-NLP Mid-Term Project: Financial Product Complaint Classification and Summarization**

## **Business Context**

### **Description**
*In the modern financial industry, customer complaints play a crucial role in identifying areas where financial institutions can improve their services. Effectively categorizing these complaints into specific product categories, such as credit reports, student loans, or money transfers, is essential for addressing customer concerns promptly by routing the tickets to relevant personnel. Leveraging Generative AI for text classification can help financial institutions better understand customer grievances and respond more efficiently. Apart from this, a summary of the customer complaint helps the support personnel quickly grasp the gist of the grievance*

### **Objective**
*The primary goal of this project is to utilize Generative AI techniques to improve the classification and summarization of customer complaints in the financial sector.
Specifically, the project will focus on:*

1. **Text-to-Label Classification:** *Implementing Zero-shot and Few-shot prompting methods to accurately classify customer complaints into relevant product categories.*
2. **Text-to-Text Summarization:** *Using Zero-shot prompting to generate concise summaries of customer complaints, enabling more personalized and effective responses.*

### **Conclusion**
*Upon completing this project, you will have the capability to develop solutions for LLM-based text classification and summarization. These tools will enable financial institutions to automate the complaint handling process, leading to faster, more accurate responses to customer issues, improved customer satisfaction, and enhanced compliance with industry regulations. This project will also provide you with valuable skills and experience that can be applied to a range of real-world business challenges.*


## changing home directory

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/GL-GenAI/Projects')
os.listdir()

['Complains_classification.csv',
 'Learners_Notebook_Mid_Term_Project.ipynb',
 'new_data_cleaned.csv',
 'requirement.txt',
 'SM-V0-Learners_Notebook_Mid_Term_Project.ipynb']

## verifying GPU

In [3]:
!nvidia-smi

Tue Jan  7 05:14:52 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# **Section 1 : Setting Up for Prompt Engineering with Mistral Model**

### **Install & Importing neccessary libraries**

- **ninja-build**: A fast build system widely used for compiling code in projects such as TensorFlow or PyTorch.
- **cmake**: A cross-platform build system generator used to manage the build process of software.
- **ipywidgets**: A package for creating interactive widgets in Jupyter Notebooks (and Google Colab) -- disabling because not needed for what we have here

In [67]:
%%capture
!apt-get update
!apt-get install -y ninja-build cmake
!pip install ipywidgets --upgrade

In [5]:
import torch

In [6]:
# This part of code will skip all the un-necessary warnings which can occur during the execution of this project.
import warnings
warnings.filterwarnings("ignore", category=Warning)

In [7]:
# Installation for GPU llama-cpp-python==0.2.69
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python==0.2.69
# For downloading the models from HF Hub
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 MB 48.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.0 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.69-cp310-cp310-linux_x86_64.whl size=55279198 sha256=9bbc005d6c550c45bbd74d8684ab1cd87c03f814ae2abf6a2e629f5a831c9984
  Stored in directory: /root/.cache/pip/wheels/e4/18/46/58b5c613b17c8d000d79ae650980fe871b3b490e04e6faa1c1
Successfully built llama-cpp-python


In [9]:
%%capture
!pip install evaluate
!pip install bert-score

In [10]:
!pip freeze > requirement.txt

In [11]:
# Basic Imports for Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import re

import torch
import evaluate

# from google.colab import drive
import locale

### **Question 1: Importing Libaries and Mistral Model (3 Marks)**

- For the Mistral Model name or path and model basename, refer to the **Week 3 Additional Content: Prompt Engineering Fundamentals**
- Code Notebook: Self-Consistency and Tree-of-Thought Prompting with Llama 2 and Mistral.





https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/blob/main/mistral-7b-instruct-v0.2.Q5_K_M.gguf

#### **why use Llama wrapper for Mistral model:**

The Llama wrapper from the llama_cpp library is being used for the Mistral model because the Mistral architecture is compatible with the Llama 2 format. Specifically:

- **Why Use Llama for Mistral?**
	1.	**Mistral is Llama-Compatible:**
	•	The Mistral model architecture was designed to be compatible with the Llama framework in terms of tokenization, model weights, and processing. This compatibility allows Mistral models to be loaded and run using libraries built for Llama, such as llama_cpp.
	2.	**Shared Ecosystem:**
	•	Mistral builds on advances introduced in the Llama ecosystem, including attention mechanisms and optimization techniques. As a result, Mistral models can take advantage of the tools and utilities developed for Llama-based models.
	3.	**Convenience:**
	•	By using llama_cpp, you can leverage its high-performance inference capabilities, such as quantization and support for GPUs, without needing a separate library for Mistral.
	4.	**GGUF Format:**
	•	The TheBloke/Mistral-7B-Instruct-v0.2-GGUF model is distributed in GGUF format, which is a format designed for compatibility with the Llama inference framework (llama_cpp). This ensures seamless integration with the Llama wrapper.
	5.	**Flexibility and Optimizations:**
	•	The llama_cpp library provides numerous options for optimizing inference (e.g., n_gpu_layers, n_batch, and quantized formats like Q5_K_M), which are useful for running Mistral efficiently.

In [12]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [13]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q5_K_M.gguf"

In [14]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

mistral-7b-instruct-v0.2.Q5_K_M.gguf:   0%|          | 0.00/5.13G [00:00<?, ?B/s]

In [15]:
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=4096 # Context window
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loa

- CUDA0 KV buffer size = 512.00 MiB
- CUDA0 compute buffer size = 296.00 MiB
- computation graph is set up and optimized for GPU execution
  - graph nodes = 1030
  - graph splits = 2
- CUDA_Host output buffer size = 0.12 MiB  

# **Section 2: Text to Label generation**

### **Question 2: Zero-Shot Prompting for Text Classification (5 Marks)**

##### **Q2.1: Define the Prompt Template, System Message, generate_prompt** **(2 Marks)**

- Define a **system message** as a string and assign it to the variable system_message to generate product class.
- Create a **zero shot prompt template** that incorporates the system message and user input.
- Define **generate_prompt** function that takes both the system_message and user_input as arguments and formats them into a prompt template


Write a Python function called **generate_mistral_response** that takes a single parameter, narrative, which represents the user's complain. Inside the function, you should perform the following tasks:


- **Combine the system_message and narrative to create a prompt string using generate_prompt function.**

*Generate a response from the Mistral model using the lcpp_llm instance with the following parameters:*

- prompt should be the combined prompt string.
- max_tokens should be set to 1200.
- temperature should be set to 0.
- top_p should be set to 0.95.
- repeat_penalty should be set to 1.2.
- top_k should be set to 50.
- stop should be set as a list containing '/s'.
- echo should be set to False.
Extract and return the response text from the generated response.

Don't forget to provide a value for the system_message variable before using it in the function.

In [16]:
system_message = """You are a product MultiLabel categorizer. Your task is to analyze product descriptions
and categorize them into the most relavant class (MultiLabel Categorization).
choose from the following categories: credit_card, retail_banking, credit_reporting, mortgages_and_loans, debt_collection
Do not explain your answer or give justifications.

give your answer in the following format:
category: answer

"""


In [17]:
zero_shot_prompt_template = """<s>[INST] {system_message}

User Input: {user_message}

Please classify this product. [/INST]"""

In [18]:
def generate_prompt(system_message, user_input):
    prompt = zero_shot_prompt_template.format(
        system_message=system_message,
        user_message=user_input
    )
    return prompt

In [19]:
def generate_mistral_response(input_text):

    # Combine user_prompt and system_message to create the prompt
    prompt = generate_prompt(system_message,input_text)

    # Define the Llama model along with its parameters for generating a response
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=1200,
        temperature=0.0,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=['/s'],
        echo=False
    )

    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    print(response_text)
    return response_text

In [20]:
# Load a CSV File containing Dataset of 500 products, narrative and summary (summary of narrative)
data=pd.read_csv('Complains_classification.csv')

In [21]:
# Randomly select 30 rows
new_data = data.sample(n=30, random_state=40)

#### **take a quick look at new_data format**

In [22]:
new_data.head(3)

,product,narrative,summary
167,retail_banking,fraudulent charge totaling made capital one ch...,A fraudulent charge was made on the individual...
169,credit_reporting,block except otherwise provided section consum...,The text outlines various stipulations regardi...
461,credit_card,usaa master plan collect cancellation debt usa...,The input appears to be a complaint about USAA...


In [23]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, 167 to 463
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   product    30 non-null     object
 1   narrative  30 non-null     object
 2   summary    30 non-null     object
dtypes: object(3)
memory usage: 960.0+ bytes


In [24]:
# show counts of product column
new_data['product'].value_counts()

,count
product,
credit_reporting,22
credit_card,4
mortgages_and_loans,3
retail_banking,1


In [25]:
data['product'].value_counts()

,count
product,
credit_reporting,388
mortgages_and_loans,36
debt_collection,29
credit_card,28
retail_banking,19


In [26]:
new_data.columns

Index(['product', 'narrative', 'summary'], dtype='object')

##### **Q2.2: Create a new column in the DataFrame called 'mistral_response' and populate it with responses generated by applying the 'generate_mistral_response' function to each 'narrative' in the DataFrame and prepare the mistral_response_cleaned column using extract_category function** **(1 Marks)**

- **progress_apply Explanation**:
	- progress_apply is a method provided by the tqdm library, specifically when used with a Pandas DataFrame or Series.
	- It extends the functionality of apply, which is used to apply a function to each element of a Pandas Series or DataFrame column.
	- progress_apply adds a progress bar to give visual feedback on the progress of the operation, which is especially useful for long-running tasks.

In [27]:
# # example - new_data['mistral_response'] = new_data['narrative'].apply(lambda x:______ )
# new_data['mistral_response'] = new_data['narrative'].apply(lambda x: "_____")

In [28]:
# Add progress bar and error handling for model responses
tqdm.pandas(desc="Generating classifications")
new_data['mistral_response'] = new_data['narrative'].progress_apply(
    lambda x: generate_mistral_response(x) if isinstance(x, str) else "Invalid input"
)


Generating classifications:   0%|          | 0/30 [00:00<?, ?it/s]
llama_print_timings:        load time =     541.47 ms
llama_print_timings:      sample time =       8.81 ms /    15 runs   (    0.59 ms per token,  1703.38 tokens per second)
llama_print_timings: prompt eval time =     541.02 ms /   330 tokens (    1.64 ms per token,   609.96 tokens per second)
llama_print_timings:        eval time =     364.19 ms /    14 runs   (   26.01 ms per token,    38.44 tokens per second)
llama_print_timings:       total time =     951.39 ms /   344 tokens
Generating classifications:   7%|▋         | 2/30 [00:00<00:13,  2.08it/s]Llama.generate: prefix-match hit


 category: credit_card, credit\_reporting, debt\_collection



llama_print_timings:        load time =     541.47 ms
llama_print_timings:      sample time =       4.13 ms /     7 runs   (    0.59 ms per token,  1696.56 tokens per second)
llama_print_timings: prompt eval time =     475.46 ms /   512 tokens (    0.93 ms per token,  1076.86 tokens per second)
llama_print_timings:        eval time =     164.23 ms /     6 runs   (   27.37 ms per token,    36.53 tokens per second)
llama_print_timings:       total time =     663.12 ms /   518 tokens
Generating classifications:  10%|█         | 3/30 [00:01<00:15,  1.78it/s]Llama.generate: prefix-match hit


 category: credit_reporting



llama_print_timings:        load time =     541.47 ms
llama_print_timings:      sample time =       3.46 ms /     6 runs   (    0.58 ms per token,  1736.61 tokens per second)
llama_print_timings: prompt eval time =     301.83 ms /   189 tokens (    1.60 ms per token,   626.17 tokens per second)
llama_print_timings:        eval time =     133.92 ms /     5 runs   (   26.78 ms per token,    37.34 tokens per second)
llama_print_timings:       total time =     454.49 ms /   194 tokens
Generating classifications:  13%|█▎        | 4/30 [00:02<00:13,  1.90it/s]Llama.generate: prefix-match hit


...

 category: credit_reporting



llama_print_timings:        load time =     541.47 ms
llama_print_timings:      sample time =       3.94 ms /     7 runs   (    0.56 ms per token,  1776.65 tokens per second)
llama_print_timings: prompt eval time =     485.34 ms /   512 tokens (    0.95 ms per token,  1054.93 tokens per second)
llama_print_timings:        eval time =     167.90 ms /     6 runs   (   27.98 ms per token,    35.73 tokens per second)
llama_print_timings:       total time =     674.78 ms /   518 tokens
Generating classifications: 100%|██████████| 30/30 [00:16<00:00,  1.83it/s]Llama.generate: prefix-match hit


 category: credit_reporting



llama_print_timings:        load time =     541.47 ms
llama_print_timings:      sample time =       3.85 ms /     7 runs   (    0.55 ms per token,  1818.65 tokens per second)
llama_print_timings: prompt eval time =     199.45 ms /   102 tokens (    1.96 ms per token,   511.42 tokens per second)
llama_print_timings:        eval time =     157.94 ms /     6 runs   (   26.32 ms per token,    37.99 tokens per second)
llama_print_timings:       total time =     378.35 ms /   108 tokens
Generating classifications: 100%|██████████| 30/30 [00:17<00:00,  1.74it/s]

 category: credit_reporting


In [29]:
new_data.head(12)

,product,narrative,summary,mistral_response
167,retail_banking,fraudulent charge totaling made capital one ch...,A fraudulent charge was made on the individual...,"category: credit_card, credit\_reporting, deb..."
169,credit_reporting,block except otherwise provided section consum...,The text outlines various stipulations regardi...,category: credit_reporting
461,credit_card,usaa master plan collect cancellation debt usa...,The input appears to be a complaint about USAA...,category: debt_collection
253,credit_reporting,block except otherwise provided section consum...,The text pertains to the stipulations and oper...,category: credit_reporting
42,credit_reporting,open account acct opened balance account acct ...,The input is about various accounts being open...,category: retail_banking
369,credit_reporting,except otherwise provided section consumer rep...,This legal text details rules and regulations ...,category: credit_reporting
26,mortgages_and_loans,trying close loan month provided income paymen...,The individual has been attempting to close a ...,category: mortgages_and_loans
377,credit_reporting,except otherwise provided section consumer rep...,This legislation requires that a consumer repo...,category: credit_reporting
238,credit_reporting,block except otherwise provided section consum...,The section explains the rules and procedures ...,category: credit_reporting
374,credit_reporting,except otherwise provided section consumer rep...,The input text seems to discuss a law provisio...,category: credit_reporting


In [30]:
def extract_category(text):
    # Define the regex pattern to match "category:" or "Category:" followed by a word
    pattern = r'category:\s*(\w+)'  # The pattern itself remains the same

    # Use re.search with the re.IGNORECASE flag to make it case-insensitive
    match = re.search(pattern, text, re.IGNORECASE)

    # If a match is found, return the captured group, else return None
    if match:
        return match.group(1)
    else:
        pattern1 = r'(credit_card|retail_banking|credit_reporting|mortgages_and_loans|debt_collection)'
        match = re.search(pattern1, text, re.IGNORECASE)
        if match:
            return match.group()
        else:
            return ''

In [31]:
# example - new_data['mistral_response_cleaned'] = new_data['narrative'].apply(lambda x:______ )
new_data['mistral_response_cleaned'] = new_data['mistral_response'].apply(
    lambda x: extract_category(x) if isinstance(x, str) else "Invalid input"
)

### Note: Match group 1 matches the 1st match only

In [32]:
new_data.head(3)

,product,narrative,summary,mistral_response,mistral_response_cleaned
167,retail_banking,fraudulent charge totaling made capital one ch...,A fraudulent charge was made on the individual...,"category: credit_card, credit\_reporting, deb...",credit_card
169,credit_reporting,block except otherwise provided section consum...,The text outlines various stipulations regardi...,category: credit_reporting,credit_reporting
461,credit_card,usaa master plan collect cancellation debt usa...,The input appears to be a complaint about USAA...,category: debt_collection,debt_collection


#### Let's take a quick look at product, mistral_response, and mistral_response_cleaned

In [33]:
new_data['mistral_response'].value_counts()

,count
mistral_response,
category: credit_reporting,21
category: mortgages_and_loans,3
category: debt_collection,2
category: retail_banking,2
"category: credit_card, credit\_reporting, debt\_collection",1
category: credit_card,1


In [34]:
new_data['mistral_response_cleaned'].value_counts()

,count
mistral_response_cleaned,
credit_reporting,21
mortgages_and_loans,3
credit_card,2
debt_collection,2
retail_banking,2


In [35]:
new_data['product'].value_counts()

,count
product,
credit_reporting,22
credit_card,4
mortgages_and_loans,3
retail_banking,1


##### **Q2.3: Calculate the F1 score** **(1 Marks)**

**Micro-averaging** aggregates the contributions of all classes (true positives, false positives, false negatives) before calculating precision and recall. This approach is useful when the class distributions are imbalanced.

**Formulaes**

**F1 Score:**
$$
F1 = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
$$

**Precision:**
$$
\text{Precision} = \frac{\text{True Positives (TP)}}{\text{True Positives (TP)} + \text{False Positives (FP)}}
$$

**Recall**
$$
\text{Recall} = \frac{\text{True Positives (TP)}}{\text{True Positives (TP)} + \text{False Negatives (FN)}}
$$

**Micro-Averaged F1 Score**
$$
F1_{\text{micro}} = 2 \times \frac{\sum \text{TP}}{2 \times \sum \text{TP} + \sum \text{FP} + \sum \text{FN}}
$$

In [36]:
# Calculate F1 score for 'product' and 'mistral_response'
f1 =  f1_score(new_data['product'], new_data['mistral_response'],average='micro')

print(f'F1 Score: {f1}')

F1 Score: 0.0


In [37]:
# Calculate F1 score for 'product' and 'mistral_response_cleaned'
f2 =  f1_score(new_data['product'], new_data['mistral_response_cleaned'],average='micro')
print(f'F1 Score: {f2}')

F1 Score: 0.8333333333333334


##### **Q2.4: Explain the difference in F1 scores between mistral_response and mistral_response_cleaned.** **(1 Marks)**

**F1 Score for `mistral_response` (0.0):**
- A score of 0.0 indicates no overlap between the predicted (`mistral_response`) and true (`product`) categories. This suggests that:
  - The multi-category entries in `mistral_response` do not match any entries in `product`.
  - Prefixes or formatting issues (e.g., `category: credit_reporting`) prevent direct comparison.

**F1 Score for `mistral_response_cleaned` (0.8333):**
- Cleaning the data resolves these inconsistencies, leading to much better alignment between `mistral_response_cleaned` and `product`. This alignment results in a high F1 score, as the majority of categories in `mistral_response_cleaned` match those in `product`.

### **Question 3: Few-Shot Prompting for Text Classification (7 Marks)**

##### **Q3.1: Prepare examples for a few-shot prompt, formulate the prompt, and generate the Mistral response. (5 Marks)**

**Generate a set of gold examples by randomly selecting 10 instances of user_input and assistant_output from dataset ensuring a balanced representation with 2 examples from each class.**

In [38]:
import json
review_1 = data[data['product'] == 'credit_card'] # choosing only rows which have credit_card
review_2 = data[data['product'] == 'retail_banking']
review_3 = data[data['product'] == 'credit_reporting']
review_4 = data[data['product'] == 'mortgages_and_loans']
review_5 = data[data['product'] == 'debt_collection']

# Sample 2 examples for each category
examples_1 = review_1.sample(2, random_state=40)
examples_2 = review_2.sample(2, random_state=40)
examples_3 = review_3.sample(2, random_state=40)
examples_4 = review_4.sample(2, random_state=40)
examples_5 = review_5.sample(2, random_state=40)

# Concatenate examples for few shot prompting
examples_df = pd.concat([examples_1,examples_2,examples_3,examples_4,examples_5 ])

# Create the training set by excluding examples
gold_examples_df = data.drop(index=examples_df.index)

# Convert examples to JSON
columns_to_select = ['narrative', 'product']
examples_json = examples_df[columns_to_select].to_json(orient='records')

# Print the first record from the JSON
print(json.loads(examples_json)[0])

# Print the shapes of the datasets
print("Examples Set Shape:", examples_df.shape)
print("Gold Examples Shape:", gold_examples_df.shape)

{'narrative': 'called request new york state covid relief plan day interest fee waived amex provided relief leading late payment amex refused honor relief day gap insists charging late fee', 'product': 'credit_card'}
Examples Set Shape: (10, 3)
Gold Examples Shape: (490, 3)


- Define your **system_message**.
- Define **first_turn_template**, **example_template** and **prediction template**
- **create few shot prompt** using gold examples and system_message
- Randomly select 30 rows from test_df as test_data
- Create **mistral_response** with **mistral_response_cleaned** columns for this

In [39]:
system_message = """You are a product MultiLabel categorizer. Your task is to analyze product descriptions
and categorize them into the most relavant class (MultiLabel Categorization).
choose from the following categories: credit_card, retail_banking, credit_reporting, mortgages_and_loans, debt_collection
Do not explain your answer or give justifications.

give your answer in the following format:
category: answer

Correct Format for answer:
category: credit_reporting, retail_banking, debt_collection

Incorrecte Format for answer:
I. credit_reporting\nII. retail_banking (The text mentions "consumer reporting agency" multiple times, which is a common term in the context of banking and finance.)\nIII. debt_collection (There are references to "debt collection" and "debtor" throughout the text.)	credit_reporting

Some more Examples of input text to be analzyed is enclosed in triple backticks, that is, ``` and the respective categories are shown as category: answer

"""

In [40]:
first_turn_template = """<s>[INST]{system_message}```{user_input}```[/INST]{assistant_output}</s> """

In [41]:
examples_template = """<s>[INST]```{user_input}```[/INST]{assistant_output}</s>"""

In [42]:
prediction_template = """<s>[INST]```{user_input}```[/INST]</s>"""

In [43]:
def create_few_shot_prompt(system_message, examples_df):

    """
    Return a prompt message in the format expected by Mistral 7b.
    10 examples are selected randomly as golden examples to form the
    few-shot prompt.
    We then loop through each example and parse the narrative as the user message
    and the product as the assistant message.

    Args:
        system_message (str): system message with instructions for classification
        examples(DataFrame): A DataFrame with examples (product + narrative + summary)
        to form the few-shot prompt.

    Output:
        few_shot_prompt (str): A prompt string in the Mistral format
    """

    few_shot_prompt = ''

    columns_to_select = ['narrative', 'product']
    examples = (
        examples_df.loc[:, columns_to_select].to_json(orient='records')
    )

    for idx, example in enumerate(json.loads(examples)):
        user_input_example = example['narrative']
        # Prepend "category: " to the product field
        assistant_output_example = f"category: {example['product']}"

        if idx == 0:
            few_shot_prompt += first_turn_template.format(
                system_message=system_message,
                user_input=user_input_example,
                assistant_output=assistant_output_example
            )
        else:
            few_shot_prompt += examples_template.format(
                user_input=user_input_example,
                assistant_output=assistant_output_example
            )

    return few_shot_prompt

In [44]:
few_shot_prompt = create_few_shot_prompt(system_message, examples_df)

In [45]:
print(few_shot_prompt)

<s>[INST]You are a product MultiLabel categorizer. Your task is to analyze product descriptions 
and categorize them into the most relavant class (MultiLabel Categorization). 
choose from the following categories: credit_card, retail_banking, credit_reporting, mortgages_and_loans, debt_collection
Do not explain your answer or give justifications. 

give your answer in the following format:
category: answer

Correct Format for answer:
category: credit_reporting, retail_banking, debt_collection

Incorrecte Format for answer:
I. credit_reporting
II. retail_banking (The text mentions "consumer reporting agency" multiple times, which is a common term in the context of banking and finance.)
III. debt_collection (There are references to "debt collection" and "debtor" throughout the text.)	credit_reporting

Some more Examples of input text to be analzyed is enclosed in triple backticks, that is, ``` and the respective categories are shown as category: answer

```called request new york state c

In [46]:
def generate_prompt(few_shot_prompt,new_review):
    prompt =  few_shot_prompt + prediction_template.format(user_input=new_review)
    return prompt

In [47]:
def generate_mistral_response(support_ticket_text):

    # Combine user_prompt and system_message to create the prompt
    prompt = generate_prompt(system_message,support_ticket_text)

    # Define the Llama model along with its parameters for generating a response
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=1200,
        temperature=0.0,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=['/s'],
        echo=False
    )
    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    print(response_text)
    return response_text

In [48]:
# Randomly select 50 rows from gold_examples
#new_data = gold_examples_df.sample(n=50, random_state=40)
new_data = gold_examples_df.sample(n=50, random_state=40)

In [49]:
# Add progress bar and error handling for model responses
tqdm.pandas(desc="Generating classifications")
new_data['mistral_response'] = new_data['narrative'].progress_apply(
    lambda x: generate_mistral_response(x) if isinstance(x, str) else "Invalid input"
)


Generating classifications:   0%|          | 0/50 [00:00<?, ?it/s]Llama.generate: prefix-match hit

llama_print_timings:        load time =     541.47 ms
llama_print_timings:      sample time =      34.86 ms /    58 runs   (    0.60 ms per token,  1664.04 tokens per second)
llama_print_timings: prompt eval time =    1019.80 ms /   907 tokens (    1.12 ms per token,   889.39 tokens per second)
llama_print_timings:        eval time =    1605.49 ms /    57 runs   (   28.17 ms per token,    35.50 tokens per second)
llama_print_timings:       total time =    2851.29 ms /   964 tokens
Generating classifications:   4%|▍         | 2/50 [00:02<01:08,  1.43s/it]Llama.generate: prefix-match hit


 I. credit_reporting, retail_banking

II. credit\_reporting, mortgage\_and\_loans, debt\_collection (There are references to "debt collection" and "mortgages and loans," as well as "credit reporting.")



llama_print_timings:        load time =     541.47 ms
llama_print_timings:      sample time =      43.76 ms /    76 runs   (    0.58 ms per token,  1736.67 tokens per second)
llama_print_timings: prompt eval time =     451.46 ms /   434 tokens (    1.04 ms per token,   961.33 tokens per second)
llama_print_timings:        eval time =    2063.95 ms /    75 runs   (   27.52 ms per token,    36.34 tokens per second)
llama_print_timings:       total time =    2745.94 ms /   509 tokens
Generating classifications:   6%|▌         | 3/50 [00:05<01:33,  1.99s/it]Llama.generate: prefix-match hit


 category: credit_reporting

This text mentions "consumer reporting agency" multiple times, indicating that the topic is related to credit reporting. The text also discusses various procedures and requirements for blocking or unblocking consumer information in a consumer reporting agency's files due to identity theft or errors. These are common practices in the context of credit reporting agencies.



llama_print_timings:        load time =     541.47 ms
llama_print_timings:      sample time =     153.20 ms /   274 runs   (    0.56 ms per token,  1788.52 tokens per second)
llama_print_timings: prompt eval time =     223.44 ms /    19 tokens (   11.76 ms per token,    85.03 tokens per second)
llama_print_timings:        eval time =    7461.57 ms /   273 runs   (   27.33 ms per token,    36.59 tokens per second)
llama_print_timings:       total time =    8547.77 ms /   292 tokens
Generating classifications:   8%|▊         | 4/50 [00:14<03:24,  4.44s/it]Llama.generate: prefix-match hit


...

 category: credit_reporting, debt_collection

```The consumer reporting agency made an error on my credit report by marking my account forbearance without my authorization. I noticed the issue when I tried to apply for a new home loan at another bank. The missing payment history from the past month was preventing me from getting approved. I contacted the specialized loan servicing company immediately and asked them to fix the error. They provided me with a letter detailing the issue, but it took several weeks of countless emails and phone calls to get anything resolved. Despite my account never being in forbearance, they were unable to fix the payment history issue without further delaying the new loan application process. I need their help immediately to get this fixed.```



llama_print_timings:        load time =     541.47 ms
llama_print_timings:      sample time =      41.66 ms /    73 runs   (    0.57 ms per token,  1752.24 tokens per second)
llama_print_timings: prompt eval time =     532.80 ms /   422 tokens (    1.26 ms per token,   792.04 tokens per second)
llama_print_timings:        eval time =    2519.18 ms /    72 runs   (   34.99 ms per token,    28.58 tokens per second)
llama_print_timings:       total time =    3272.81 ms /   494 tokens
Generating classifications: 100%|██████████| 50/50 [04:33<00:00,  4.99s/it]Llama.generate: prefix-match hit


 category: credit_reporting

This text contains multiple references to a "consumer reporting agency" and various types of consumer information being reported or blocked. This indicates that the product descriptions are related to credit reporting services, which falls under this category. The other categories mentioned in the problem statement do not appear to be relevant based on the given input text.



llama_print_timings:        load time =     541.47 ms
llama_print_timings:      sample time =      43.88 ms /    76 runs   (    0.58 ms per token,  1731.80 tokens per second)
llama_print_timings: prompt eval time =     528.99 ms /   434 tokens (    1.22 ms per token,   820.43 tokens per second)
llama_print_timings:        eval time =    2603.80 ms /    75 runs   (   34.72 ms per token,    28.80 tokens per second)
llama_print_timings:       total time =    3373.05 ms /   509 tokens
Generating classifications: 100%|██████████| 50/50 [04:36<00:00,  5.54s/it]

 category: credit_reporting

This text mentions "consumer reporting agency" multiple times, indicating that the topic is related to credit reporting. The text also discusses various procedures and requirements for blocking or unblocking consumer information in a consumer reporting agency's files due to identity theft or errors. These are common practices in the context of credit reporting agencies.


In [50]:
# example - new_data['mistral_response_cleaned'] = new_data['narrative'].apply(lambda x:______ )
new_data['mistral_response_cleaned'] = new_data['mistral_response'].apply(
    lambda x: extract_category(x) if isinstance(x, str) else "Invalid input"
)

In [51]:
new_data.head(8)

,product,narrative,summary,mistral_response,mistral_response_cleaned
74,credit_reporting,anyone us credit report another consumer repor...,The narrator is filing a complaint against ind...,"I. credit_reporting, retail_banking\n\nII. cr...",credit_reporting
174,credit_reporting,block except otherwise provided section consum...,The input refers to regulations for consumer r...,category: credit_reporting\n\nThis text menti...,credit_reporting
85,credit_reporting,shocked reviewed credit report found late paym...,The user reviewed their credit report and was ...,category: credit_reporting\n\n```The consumer...,credit_reporting
372,credit_reporting,except otherwise provided section consumer rep...,This passage discusses the procedures and requ...,category: credit_reporting\n\nThis text conta...,credit_reporting
296,credit_reporting,block except otherwise provided section consum...,The input text discusses the obligations of co...,category: credit_reporting\n\nThis text menti...,credit_reporting
247,credit_reporting,block except otherwise provided section consum...,The document discusses rules regarding blockin...,category: credit_reporting\n\nThis text prima...,credit_reporting
386,credit_reporting,except otherwise provided section consumer rep...,This section mandates consumer reporting agenc...,category: credit_reporting\n\nThis text conta...,credit_reporting
51,debt_collection,left apartment charging u usage credence colle...,The writer has left their apartment and has be...,"category: debt_collection, credit_reporting (...",debt_collection


##### **Q3.2: Calculate the F1 score** **(1 Marks)**

In [52]:
# Calculate F1 score for 'product' and 'mistral_response_cleaned'
f1 =  f1_score(new_data['product'], new_data['mistral_response_cleaned'],average='micro')

print(f'F1 Score: {f1}')

F1 Score: 0.82


##### **Q3.3: Share your observations on the few-shot and zero-shot prompt techniques. (1 Marks)**

- Zero-Shot Prompting gave F1 score of 0.8333
- Few-Shot Prompting gave F1 score of 0.82

it is normal for Few-Shot Prompting to sometimes perform slightly worse than Zero-Shot Prompting in certain cases. The difference in F1 scores (0.8333 vs. 0.82) is relatively small and can occur due to several factors:


	1.	Overfitting to Few-Shot Examples:
	      •	The model may focus too heavily on the patterns present in the few-shot examples, which might not generalize well to the test data. This can slightly degrade performance in some cases.
	2.	Prompt Length and Context Window:
	      •	Few-shot prompting increases the prompt length by including multiple examples. If the total input exceeds the model’s context window, it might truncate the prompt or struggle to prioritize the relevant parts, leading to reduced performance.
	3.	Zero-Shot Prompt Design:
	      •	A well-crafted zero-shot prompt might provide clear instructions that the model can generalize better compared to noisy or suboptimal few-shot examples.
	4.	Randomness in Few-Shot Example Selection:
	      •	The random selection of few-shot examples may introduce variation in performance. Some examples may align better with the test data than others, leading to fluctuations in results.

# **Section 3: Text to Text generation**

### **Question 4: Zero-Shot Prompting for Text Summarization (5 Marks)**

##### **Q4.1: Define the Prompt Template, System Message, generate prompt and model response** **(3 Marks)**


- Define a **system message** as a string and assign it to the variable system_message to generate summary of narrative in data.
- Create a **zero shot prompt template** that incorporates the system message and user input.
- Define **generate_prompt** function that takes both the system_message and user_input as arguments and formats them into a prompt template


Write a Python function called **generate_mistral_response** that takes a single parameter, narrative, which represents the user's complain. Inside the function, you should perform the following tasks:


- **Combine the system_message and narrative to create a prompt string using generate_prompt function.**

*Generate a response from the Mistral model using the lcpp_llm instance with the following parameters:*

- prompt should be the combined prompt string.
- max_tokens should be set to 1200.
- temperature should be set to 0.
- top_p should be set to 0.95.
- repeat_penalty should be set to 1.2.
- top_k should be set to 50.
- stop should be set as a list containing '/s'.
- echo should be set to False.
Extract and return the response text from the generated response.

Don't forget to provide a value for the system_message variable before using it in the function.

In [53]:
system_message = """

Summarize the dialogue mentioned in the user input below. Be specific and concise in your summary.
The dialogue will be delimited by triple backticks, that is, ```.
"""

In [54]:
zero_shot_prompt_template = """<s>[INST] {system_message}

User Input: ```{user_message}```[/INST]"""

def generate_prompt(system_message,user_input):
    prompt=zero_shot_prompt_template.format(system_message=system_message,user_message=user_input)
    return prompt

def generate_mistral_response(input_text):

    # Combine user_prompt and system_message to create the prompt
    prompt = generate_prompt(system_message,input_text)

    # Define the Llama model along with its parameters for generating a response
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=1200,
        temperature=0.0,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=['/s'],
        echo=False
    )


    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    print(response_text)
    return response_text

##### **Q4.2: Generate mistral_response column containing LLM generated summaries** **(1 Marks)**

In [55]:
# Randomly select 30 rows
gold_examples = data.sample(n=30, random_state=40)


In [56]:
# example - new_data['mistral_response_cleaned'] = new_data['narrative'].apply(lambda x:______ )
# gold_examples['mistral_response'] = gold_examples['narrative'].apply(lambda x: "_____")


tqdm.pandas(desc="Generating Summaries")
gold_examples['mistral_response'] = gold_examples['narrative'].progress_apply(
    lambda x: generate_mistral_response(x) if isinstance(x, str) else "Invalid input"
)


Generating Summaries:   0%|          | 0/30 [00:00<?, ?it/s]Llama.generate: prefix-match hit

llama_print_timings:        load time =     541.47 ms
llama_print_timings:      sample time =     108.68 ms /   189 runs   (    0.58 ms per token,  1738.99 tokens per second)
llama_print_timings: prompt eval time =     428.74 ms /   270 tokens (    1.59 ms per token,   629.75 tokens per second)
llama_print_timings:        eval time =    5557.25 ms /   188 runs   (   29.56 ms per token,    33.83 tokens per second)
llama_print_timings:       total time =    6581.07 ms /   458 tokens
Generating Summaries:   7%|▋         | 2/30 [00:06<01:32,  3.30s/it]Llama.generate: prefix-match hit


 The user is describing a dispute with Capital One over a fraudulent debit card charge. They noticed the unauthorized transaction and had it cancelled immediately. Capital One disputed the claim, but provisional credit was issued pending determination. A denial of the claim followed, despite the user's insistence that they never lost possession of their debit card or authorized its use by someone else. The user received a form letter from the bank explaining the denial and withdrew their money to open a new account. They appealed the decision and had several conversations with the person making determination. The user believes their original debit card was intercepted and fraudulently activated, leading to multiple unauthorized purchases. Capital One sent them a replacement card without authorization, which they believe was used for further credit purchases without consent. The user reported the fraudulent activity to both Capital One and local police department's financial services di


llama_print_timings:        load time =     541.47 ms
llama_print_timings:      sample time =     109.48 ms /   189 runs   (    0.58 ms per token,  1726.36 tokens per second)
llama_print_timings: prompt eval time =     535.55 ms /   506 tokens (    1.06 ms per token,   944.82 tokens per second)
llama_print_timings:        eval time =    5665.99 ms /   188 runs   (   30.14 ms per token,    33.18 tokens per second)
llama_print_timings:       total time =    6800.27 ms /   694 tokens
Generating Summaries:  10%|█         | 3/30 [00:13<02:08,  4.76s/it]Llama.generate: prefix-match hit


 The dialogue outlines provisions of the Fair Credit Reporting Act regarding identity theft and blocked reporting information. When a consumer identifies potential identity theft, a consumer reporting agency shall block relevant reporting information by the business day after receiving appropriate proof from the consumer. The agency may decline or rescind the block if it determines there was no error or material misrepresentation. Consumers are to be notified promptly and in writing when their information is reinstated.

A reseller file shall apply the same rules as a consumer reporting agency, while consumers have the right to report identity theft to the bureau and obtain information regarding it from the reseller. The check service company must report identity theft to the national consumer reporting agency but cannot access blocked information without proper verification or court order. Consumers' unverified accounts must be removed if they cannot provide verifiable proof of owners


llama_print_timings:        load time =     541.47 ms
llama_print_timings:      sample time =     117.22 ms /   206 runs   (    0.57 ms per token,  1757.39 tokens per second)
llama_print_timings: prompt eval time =     299.90 ms /   183 tokens (    1.64 ms per token,   610.20 tokens per second)
llama_print_timings:        eval time =    6248.61 ms /   205 runs   (   30.48 ms per token,    32.81 tokens per second)
llama_print_timings:       total time =    7210.65 ms /   388 tokens
Generating Summaries:  13%|█▎        | 4/30 [00:20<02:27,  5.68s/it]Llama.generate: prefix-match hit


...

 The user is sharing their experience as an identity theft victim, discussing various issues they have encountered regarding fraudulent transactions appearing on their credit report from Equifax. They mention finding old accounts involved in the 2017 Equifax data breach and the long, complicated process of resolving these issues. The user expresses frustration with disputed accounts that haven't been resolved, hindering their ability to apply for new credit or even get a first-time credit card. They also mention being denied employment due to negative information on their report and undergoing an adverse action based on inaccurate reporting. The user is currently seeking help from someone named "the guy" and considering hiring an attorney specializing in predatory loans for compensation as part of the identity theft victim remedy process, which has caused significant financial and emotional stress.



llama_print_timings:        load time =     541.47 ms
llama_print_timings:      sample time =     108.81 ms /   189 runs   (    0.58 ms per token,  1737.04 tokens per second)
llama_print_timings: prompt eval time =     591.52 ms /   506 tokens (    1.17 ms per token,   855.43 tokens per second)
llama_print_timings:        eval time =    6704.15 ms /   188 runs   (   35.66 ms per token,    28.04 tokens per second)
llama_print_timings:       total time =    7912.29 ms /   694 tokens
Generating Summaries: 100%|██████████| 30/30 [03:39<00:00,  7.65s/it]Llama.generate: prefix-match hit


 The dialogue outlines provisions of the Fair Credit Reporting Act regarding identity theft and blocked reporting information. When a consumer identifies potential identity theft, a consumer reporting agency shall block relevant reporting information by the business day after receiving appropriate proof from the consumer. The agency may decline or rescind the block if it determines there was no error or material misrepresentation. Consumers are to be notified promptly and in writing when their information is reinstated.

A reseller file shall apply the same rules as a consumer reporting agency, while consumers have the right to report identity theft to the bureau and obtain information regarding it from the reseller. The check service company must report identity theft to the national consumer reporting agency but cannot access blocked information without proper verification or court order. Consumers' unverified accounts must be removed if they cannot provide verifiable proof of owners


llama_print_timings:        load time =     541.47 ms
llama_print_timings:      sample time =      94.56 ms /   163 runs   (    0.58 ms per token,  1723.68 tokens per second)
llama_print_timings: prompt eval time =     236.48 ms /    96 tokens (    2.46 ms per token,   405.95 tokens per second)
llama_print_timings:        eval time =    5711.62 ms /   162 runs   (   35.26 ms per token,    28.36 tokens per second)
llama_print_timings:       total time =    6485.84 ms /   258 tokens
Generating Summaries: 100%|██████████| 30/30 [03:45<00:00,  7.53s/it]

 The user is having an issue with a credit card where an authorized user negatively affected their credit score by activating the card and using it, despite never having used it themselves. They have contacted Experian to dispute this change but have not seen any reflection of the removal yet. The user mentions that they are in the process of refinancing their home, which requires a good credit score. Their husband has lost his job, making it difficult for them to make ends meet with just their nurse salary. They need Experian to accurately reflect the change as soon as possible so they can proceed with the loan refinance. The user expresses that they have tried disputing online but were unable to get in touch with a representative and are asking for help in contacting Experian to resolve this issue.


In [57]:
pd.set_option('display.max_colwidth', None)

In [58]:
gold_examples.head(3)

,product,narrative,summary,mistral_response
167,retail_banking,fraudulent charge totaling made capital one checking account via debit card seeing charge immediately canceled debit card informed capital one fraudulent charge disputed charge posted account issued provisional credit pending determination claim received form letter bank denying claim money withdrawn account opened appeal denial capital one bank person every contacted discus original claim make determination furthermore never lost possession debit card authorized person use capital one sent new debit card authorized via usps replace debit card set expire card sent fraudulent charge occurred never received card believe card intercepted fraudulently activated used make fraudulent purchase complained activated card several time capital one complained fraudulent charge last call dispute denial claim never noted account asked several time person making determination claim contact person making complaint malfeasance gross business practice sending replacement card without authorization letting said card activated use credit purchase without authorization fraud reported claim made contacting discus complaint making determination appealed denial capital one bank claim number also made complaint local police dept division financial service regard fraudulent charge,"A fraudulent charge was made on the individual's Capital One checking account via a debit card. The transaction was immediately disputed and the card was cancelled. Capital One issued provisional credit. However, the bank later denied the claim and withdrew the provisional credit. The individual appealed the denial and had contacted Capital One multiple times, but no one reached out to discuss the original claim or decision. The original card was never lost and no authorization was given to anyone to use it. They believe a new, replacement card sent by Capital One was intercepted and fraudulently activated to make the charge. The individual is unhappy with Capital One's handling of the situation, stating that it is grossly improper to send a replacement card and allow it to be activated without authorization. They reported the fraud, have appealed the claim denial with Capital One and have also placed a complaint with their local police department and the Division of Financial Services.","The user is describing a dispute with Capital One over a fraudulent debit card charge. They noticed the unauthorized transaction and had it cancelled immediately. Capital One disputed the claim, but provisional credit was issued pending determination. A denial of the claim followed, despite the user's insistence that they never lost possession of their debit card or authorized its use by someone else. The user received a form letter from the bank explaining the denial and withdrew their money to open a new account. They appealed the decision and had several conversations with the person making determination. The user believes their original debit card was intercepted and fraudulently activated, leading to multiple unauthorized purchases. Capital One sent them a replacement card without authorization, which they believe was used for further credit purchases without consent. The user reported the fraudulent activity to both Capital One and local police department's financial services division."
169,credit_reporting,block except otherwise provided section consumer reporting agency shall block reporting information file consumer consumer identifies information resulted alleged identity theft later business day date receipt agency appropriate proof identity consumer copy identity theft report identification information consumer statement consumer information information relating transaction consumer b notification consumer reporting agency shall promptly notify furnisher information identified consumer subsection section information may result identity theft identity theft report filed block requested section effective date block c a

##### **Q4.3: Evaluate bert score** **(1 Marks)**

In [59]:
def evaluate_score(test_data, scorer, bert_score=False):

    """
    Return the ROUGE score or BERTScore for predictions on gold examples
    For each example we make a prediction using the prompt.
    Gold summaries and the AI generated summaries are aggregated into lists.
    These lists are used by the corresponding scorers to compute metrics.
    Since BERTScore is computed for each candidate-reference pair, we take the
    average F1 score across the gold examples.

    Args:
        prompt (List): list of messages in the Open AI prompt format
        gold_examples (str): JSON string with list of gold examples
        scorer (function): Scorer function used to compute the ROUGE score or the
                           BERTScore
        bert_score (boolean): A flag variable that indicates if BERTScore should
                              be used as the metric.

    Output:
        score (float): BERTScore or ROUGE score computed by comparing model predictions
                       with ground truth
    """

    model_predictions = test_data['mistral_response'].tolist()
    ground_truths = test_data['summary'].tolist()
    if bert_score:
        score = scorer.compute(
            predictions=model_predictions,
            references=ground_truths,
            lang="en",
            rescale_with_baseline=True # Rescale scores for better interpretability
        )

        return sum(score['f1'])/len(score['f1'])
    else:
        return scorer.compute(
            predictions=model_predictions,
            references=ground_truths
        )

In [60]:
bert_scorer = evaluate.load("bertscore")

In [61]:
score = evaluate_score(gold_examples,
    bert_scorer,
    bert_score=True)

print(f'BERTScore: {score}')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore: 0.339604996641477


## Try the Rouge Score

In [62]:
!pip install rouge-score -q

  Preparing metadata (setup.py) ... done


In [63]:
# Load a different scorer, e.g., ROUGE
rouge_scorer = evaluate.load("rouge")

score = evaluate_score(gold_examples,
    rouge_scorer,
    bert_score=False)

print(f'RougeScore: {score}')

RougeScore: {'rouge1': 0.4883015684831733, 'rouge2': 0.1539193312686402, 'rougeL': 0.2690705600894815, 'rougeLsum': 0.2972254319430034}


# Submission

In [99]:
%%capture
!apt-get update && apt-get install -y jq


In [102]:
%%shell
jq -M 'del(.metadata.widgets)' SM-V0-Learners_Notebook_Mid_Term_Project.ipynb > SM-V0-Learners_Notebook_Mid_Term_Project_fixed.ipynb


In [103]:
!jupyter nbconvert SM-V0-Learners_Notebook_Mid_Term_Project_fixed.ipynb --to html



[NbConvertApp] Converting notebook SM-V0-Learners_Notebook_Mid_Term_Project_fixed.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 1 image(s).
[NbConvertApp] Writing 670674 bytes to SM-V0-Learners_Notebook_Mid_Term_Project_fixed.html


In [94]:
os.listdir()

['Complains_classification.csv',
 'Learners_Notebook_Mid_Term_Project.ipynb',
 'new_data_cleaned.csv',
 'requirement.txt',
 'SM-V0-Learners_Notebook_Mid_Term_Project.ipynb']